<a href="https://colab.research.google.com/github/SleepyAIBot/GenAIChatBot/blob/main/ChatBotWithGeminiLangChainAndElevenLabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai
!pip install elevenlabs
!pip install langchain
!pip install langchain-core
!pip install langchain-community
!pip install langchain-google-genai
!pip install gradio
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [ ]:
import os
import re
import requests
import json
import gradio as gr
# Google Gemini imports
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
# legacy chains + memory now live in langchain_classic
from langchain_classic.chains import LLMChain
from langchain_classic.memory import ConversationBufferMemory
# ElevenLabs imports
from elevenlabs.client import ElevenLabs
from elevenlabs import save

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


**Generate API Key**

- Go to [https://aistudio.google.com/app/api-keys](https://www.google.com/url?q=https%3A%2F%2Faistudio.google.com%2Fapp%2Fapi-keys)
- Create a new Secret Key
- Copy the Secret Key for your use.

- **ElevenLabs API Key**
    - Sign up at [https://elevenlabs.io/](https://www.google.com/url?q=https%3A%2F%2Felevenlabs.io%2F)
    - Go to "Developer" section
    - Copy your API key from the "API Key" section

- Replace your Gemini AI API Key and ElevenLabs API Key with your own API Key

- **Assign your Voice to a variable and set the URL to a variable**
    - For Default/Pre-made Voices:
        1. Login to ElevenLabs at [**https://elevenlabs.io/**](https://elevenlabs.io/)
        2. Go to the Voices section from the left sidebar
        3. Click on the "Default Voices" tab
        4. Browse through the available voices (Rachel, Sarah, Clyde, etc.)
        5. Click on the three dots menu (⋮) next to any voice you want to use
        6. Select "Copy voice ID" from the dropdown menu
        7. The Voice ID will be copied to your clipboard
        8. Paste this ID as the value for `ELEVENLABS_VOICE_ID` in your code

In [ ]:
# Google Gemini API Key
GEMINI_API_KEY = ""

# ElevenLabs API Key
ELEVENLABS_API_KEY = ""

# ElevenLabs Voice ID (Rachel voice by default)
ELEVENLABS_VOICE_ID = "21m00Tcm4TlvDq8ikWAM"

# Configure Gemini
genai.configure(api_key=GEMINI_API_KEY)

# Initialize ElevenLabs client
elevenlabs_client = ElevenLabs(api_key=ELEVENLABS_API_KEY)

print("✅ API keys configured successfully!")

✅ API keys configured successfully!


In [ ]:
template = """You are a helpful assistant to answer user queries.
{chat_history}
User: {user_message}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "user_message"],
    template=template
)

memory = ConversationBufferMemory(memory_key="chat_history")

print("✅ Prompt template created!")

✅ Prompt template created!


/tmp/ipython-input-2322140030.py:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [ ]:
# Configure the Gemini model directly
gemini_model = genai.GenerativeModel('gemini-2.5-flash')

# Create a custom LLM wrapper for LangChain compatibility
class GeminiLLM:
    def __init__(self, model):
        self.model = model
        self.memory_history = []

    def predict(self, user_message):
        # Build conversation context
        full_prompt = f"You are a helpful assistant to answer user queries.\n"
        for msg in self.memory_history:
            full_prompt += f"{msg}\n"
        full_prompt += f"User: {user_message}\nChatbot:"

        # Generate response
        response = self.model.generate_content(full_prompt)
        answer = response.text

        # Update memory
        self.memory_history.append(f"User: {user_message}")
        self.memory_history.append(f"Chatbot: {answer}")

        # Keep only last 10 exchanges to avoid token limits
        if len(self.memory_history) > 20:
            self.memory_history = self.memory_history[-20:]

        return answer

In [ ]:
# Initialize Gemini model using direct Google GenerativeAI (NOT LangChain wrapper)
import google.generativeai as genai

# Configure the Gemini model directly
gemini_model = genai.GenerativeModel('gemini-2.5-flash')

# Create a custom LLM wrapper for LangChain compatibility
class GeminiLLM:
    def __init__(self, model):
        self.model = model
        self.memory_history = []

    def predict(self, user_message):
        # Build conversation context
        full_prompt = f"You are a helpful assistant to answer user queries.\n"
        for msg in self.memory_history:
            full_prompt += f"{msg}\n"
        full_prompt += f"User: {user_message}\nChatbot:"

        # Generate response
        response = self.model.generate_content(full_prompt)
        answer = response.text

        # Update memory
        self.memory_history.append(f"User: {user_message}")
        self.memory_history.append(f"Chatbot: {answer}")

        # Keep only last 10 exchanges to avoid token limits
        if len(self.memory_history) > 20:
            self.memory_history = self.memory_history[-20:]

        return answer

# Initialize the custom LLM
llm_chain = GeminiLLM(gemini_model)

print("✅ Gemini LLM initialized with direct SDK!")

✅ Gemini LLM initialized with direct SDK!


In [ ]:
# Initialize the custom LLM
llm_chain = GeminiLLM(gemini_model)

print("✅ Gemini LLM initialized with direct SDK!")

def generate_audio_elevenlabs(text):
    """
    Generate audio using ElevenLabs API
    Returns audio file path or error message
    """
    try:
        # Generate audio
        audio = elevenlabs_client.generate(
            text=text,
            voice=ELEVENLABS_VOICE_ID,
            model="eleven_monolingual_v1"  # or "eleven_multilingual_v2"
        )

        # Save audio to file
        output_path = f"/content/output_audio_{hash(text) % 10000}.mp3"
        save(audio, output_path)

        return {
            "type": "SUCCESS",
            "response": output_path,
            "message": "Audio generated successfully"
        }
    except Exception as e:
        return {
            "type": "ERROR",
            "response": str(e),
            "message": f"Audio generation failed: {str(e)}"
        }

✅ Gemini LLM initialized with direct SDK!


In [ ]:
def generate_audio_elevenlabs_http(text):
    """
    Alternative method using direct HTTP API calls
    More reliable for some use cases
    """
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{ELEVENLABS_VOICE_ID}"

    headers = {
        "Accept": "audio/mpeg",
        "Content-Type": "application/json",
        "xi-api-key": ELEVENLABS_API_KEY
    }

    data = {
        "text": text,
        "model_id": "eleven_monolingual_v1",
        "voice_settings": {
            "stability": 0.5,
            "similarity_boost": 0.5,
            "style": 0.5,
            "use_speaker_boost": True
        }
    }

    try:
        response = requests.post(url, json=data, headers=headers)
        response.raise_for_status()

        # Save audio file
        output_path = f"/content/output_audio_{hash(text) % 10000}.mp3"
        with open(output_path, 'wb') as f:
            f.write(response.content)

        return {
            "type": "SUCCESS",
            "response": output_path,
            "message": "Audio generated successfully"
        }
    except requests.exceptions.RequestException as e:
        return {
            "type": "ERROR",
            "response": str(e),
            "message": f"Audio generation failed: {str(e)}"
        }

print("✅ ElevenLabs audio functions defined!")

✅ ElevenLabs audio functions defined!


In [ ]:
def get_audio_reply_for_question(text):
    """
    Generate audio for the chatbot response
    """
    generated_audio_event = generate_audio_elevenlabs(text)

    final_response = {
        "audio_path": '',
        "message": ''
    }

    if generated_audio_event["type"] == "SUCCESS":
        audio_path = generated_audio_event["response"]
        final_response['audio_path'] = audio_path
        final_response['message'] = "Audio generated successfully"
    else:
        final_response['message'] = generated_audio_event['message']

    return final_response

print("✅ Audio reply function defined!")

✅ Audio reply function defined!


In [ ]:
def get_text_response(user_message):
    """
    Get text response from Gemini
    """
    try:
        response = llm_chain.predict(user_message=user_message)
        return response
    except Exception as e:
        error_msg = f"Error in Gemini response: {str(e)}"
        print(error_msg)
        return f"Sorry, I encountered an error: {str(e)}"

print("✅ Text response function defined!")

✅ Text response function defined!


In [ ]:
def get_text_response_and_audio_response(user_message):
    """
    Get both text response from Gemini and audio from ElevenLabs
    """
    # Get text response from Gemini
    text_response = get_text_response(user_message)

    # Generate audio for the response
    audio_reply = get_audio_reply_for_question(text_response)

    final_response = {
        'text': text_response,
        'audio_path': audio_reply.get('audio_path', ''),
        'message': audio_reply.get('message', '')
    }

    return final_response

print("✅ Combined response function defined!")

✅ Combined response function defined!


In [ ]:
def chat_bot_response(message, history):
    """
    Main chatbot function for Gradio interface
    Returns tuple of (text_response, audio_file_path)
    """
    try:
        # Get text and audio response
        response = get_text_response_and_audio_response(message)

        text_response = response['text']
        audio_path = response['audio_path']

        if audio_path and os.path.exists(audio_path):
            # Return both text and audio
            return text_response
        else:
            # Return only text if audio fails
            return text_response

    except Exception as e:
        error_msg = f"Error: {str(e)}"
        print(error_msg)
        return error_msg

print("✅ Chatbot response handler defined!")

✅ Chatbot response handler defined!


In [ ]:
demo = gr.ChatInterface(
    fn=chat_bot_response,
    title="🤖 Gemini + ElevenLabs Chatbot",
    description="Chat with Google Gemini AI with voice responses from ElevenLabs",
    examples=[
        "How are you doing?",
        "What are your interests?",
        "Tell me a short story",
        "What's the weather like today?",
        "Explain quantum computing in simple terms"
    ],
    theme=gr.themes.Soft()
)

print("✅ Gradio interface created!")

if __name__ == "__main__":
    # Launch with public link
    demo.launch(
        share=True,  # Creates public link
        debug=True   # Shows errors and logs
    )


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


✅ Gradio interface created!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2798085e092b92a645.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Error in Gemini response: 
  No API_KEY or ADC found. Please either:
    - Set the `GOOGLE_API_KEY` environment variable.
    - Manually pass the key with `genai.configure(api_key=my_api_key)`.
    - Or set up Application Default Credentials, see https://ai.google.dev/gemini-api/docs/oauth for more information.


### Publish your code with below steps

---



In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

In [ ]:
HUGGING_FACE_REPO_ID = "<<Hugging Face UserName/Repo ID>>"

Adding Secret Variables in Hugging Face Account:

- Open your Space
- Click on Settings Button
- Checkout to Variables and secrets section
- Create New Secrets

In [ ]:
%mkdir /content/ChatBotWithGeminiLangChainAndElevenLabs
!wget -P  /content/ChatBotWithGeminiLangChainAndElevenLabs/ https://s3.ap-south-1.amazonaws.com/cdn1.ccbp.in/GenAI-Workshop/ChatBotWithGeminiLangChainElevenLabs/app.py
!wget -P /content/ChatBotWithGeminiLangChainAndElevenLabs/ https://s3.ap-south-1.amazonaws.com/cdn1.ccbp.in/GenAI-Workshop/ChatBotWithGeminiLangChainElevenLabs/requirements.txt

In [ ]:
%cd /content/ChatBotWithGeminiLangChainAndElevenLabs

api.upload_file(
    path_or_fileobj="./requirements.txt",
    path_in_repo="requirements.txt",
    repo_id=HUGGING_FACE_REPO_ID,
    repo_type="space")

api.upload_file(
    path_or_fileobj="./app.py",
    path_in_repo="app.py",
    repo_id=HUGGING_FACE_REPO_ID,
    repo_type="space")